# promptx

A framework for building AI systems.

```bash
pip install pxx
```

In [1]:
from promptx import prompt

character = 'Batman'
prompt(f'Write a character profile for {character}')

2023-11-01 09:37:23.997 | INFO     | promptx:load:104 - loading local app from /home/rjl/promptx
2023-11-01 09:37:24.000 | INFO     | promptx:load:107 - loaded environment variables from /home/rjl/promptx/.env
2023-11-01 09:37:24.001 | INFO     | promptx:load:108 - API KEY wMeGC


'Name: Batman\n\nAliases: Bruce Wayne\n\nOccupation: Crime-fighter\n\nGender: Male\n\nAge: Mid-30s\n\nBackground: Bruce Wayne, a billionaire playboy philanthropist, witnessed the murder of his parents as a child. This traumatic event drove him to swear an oath to protect Gotham City from crime. He trained extensively both physically and mentally, studying various martial arts, criminology, and detective skills. He also utilizes his vast resources to develop advanced technology and equipment to aid in his fight against crime.\n\nPhysical Appearance: Batman is tall and muscular, standing at around 6\'2" and weighing approximately 210 pounds. He has a strong jawline, rugged features, and intense, piercing blue eyes. He wears a black, armored suit with a bat symbol on his chest, along with a cape and cowl that conceal his identity.\n\nPersonality: Batman is known for his brooding and serious demeanor. He is driven by a sense of justice and a desire to protect the innocent. He is highly dis

By default, this returns a plain string response, but to generate complex data you can pass in the expected schema along with the prompt input.

*Note: `Entity` is a thin layer on top of `pydantic.BaseModel` that allows the object to be stored as an embedding. You can use `pydantic.BaseModel` directly if you don't need to store the object as an embedding and just want to use it as the prompt output schema.*

In [2]:
from pydantic import Field
from promptx.collection import Entity

class Character(Entity):
    name: str = Field(..., embed=False),
    description: str = Field(..., description='Describe the character in a few sentences')
    age: int = Field(..., ge=0, le=120)

batman = prompt('Generate a character profile for Batman', output=Character)
batman

/home/rjl/promptx/venv/lib/python3.10/site-packages/pydantic/json_schema.py:2075: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, json_schema_extra={'embed': False}),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)



Character(
    id='3a05fe57-ab86-4979-b473-d5e56f911bfa',
    type='character',
    name='Batman',
    description='Batman is a fictional superhero who appears in American comic books published by DC Comics. He is a wealthy industrialist and philanthropist who dedicates his life to fighting crime in Gotham City.',
    age=42
)

This returns an instance of the specified schema using the generated response as the input data. Let's create a list of instead.

In [3]:
characters = prompt(
    'Generate some characters from the Batman universe',
    output=[Character],
)

characters

/home/rjl/promptx/venv/lib/python3.10/site-packages/pydantic/json_schema.py:2075: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, json_schema_extra={'embed': False}),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)


,id,type,name,description,age
0,8300036f-0902-4294-b91d-78d21e43f5ff,character,Batman,Batman is a superhero who is also known as Bru...,35
1,571338d9-1634-4532-be6c-000142ca4f8f,character,Joker,The Joker is a supervillain and the archenemy ...,42
2,1deddd8d-93f8-4d26-bf25-5b96ab01774c,character,Catwoman,Catwoman is a skilled criminal and anti-hero w...,30


If the output is a list, `prompt` returns a `Collection`, which extends `pd.DataFrame`. To extract the `Entity` representations, use the `objects` property.

We can now store these generated objects as embeddings in a collection.

In [4]:
from promptx import store

store(*characters.objects)

This stores the object as an embedding, along with some metadata, in a vector database (ChromaDB by default). The process is quite simple, it embeds the whole object as a JSON string and each field individually. This allows us to query the database using any field in the object.

In [5]:
from promptx import query

query()

,id,type,name,description,age
0,8300036f-0902-4294-b91d-78d21e43f5ff,character,Batman,Batman is a superhero who is also known as Bru...,35
1,571338d9-1634-4532-be6c-000142ca4f8f,character,Joker,The Joker is a supervillain and the archenemy ...,42
2,1deddd8d-93f8-4d26-bf25-5b96ab01774c,character,Catwoman,Catwoman is a skilled criminal and anti-hero w...,30


Now let's generate some more characters and add them to the collection. We'll first get any existing characters and extract their names, which we can pass to the prompt to avoid generating duplicates. Any characters generated will be added the list during iteration. Finally, we'll store all the generated characters in the collection.

In [7]:
n = 3
characters = query().objects

for _ in range(n):
    characters += prompt(
        '''
        Generate a list of new characters from the Batman universe.
        Don't use any of the existing characters.
        ''',
        input = {
            'existing_characters': [c.name for c in characters],
        },
        output=[Character],
    ).objects

store(*characters)
query()

/home/rjl/promptx/venv/lib/python3.10/site-packages/pydantic/json_schema.py:2075: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, json_schema_extra={'embed': False}),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
/home/rjl/promptx/venv/lib/python3.10/site-packages/pydantic/json_schema.py:2075: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, json_schema_extra={'embed': False}),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
/home/rjl/promptx/venv/lib/python3.10/site-packages/pydantic/json_schema.py:2075: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, json_schema_extra={'embed': False}),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnin

,id,type,name,description,age
0,8300036f-0902-4294-b91d-78d21e43f5ff,character,Batman,Batman is a superhero who is also known as Bru...,35
1,571338d9-1634-4532-be6c-000142ca4f8f,character,Joker,The Joker is a supervillain and the archenemy ...,42
2,1deddd8d-93f8-4d26-bf25-5b96ab01774c,character,Catwoman,Catwoman is a skilled criminal and anti-hero w...,30
3,400358e0-7249-4b69-932f-1b494b6aca13,character,Nightshade,Nightshade is a mysterious vigilante with the ...,30
4,29ad847d-d5a6-4929-93dd-691d56b3a69f,character,Crimson Mask,Crimson Mask is a former circus performer turn...,35
5,c15b289a-863a-4ee8-8c9a-04f1864a340a,character,Scarlet Raven,Scarlet Raven is a highly skilled hacker and i...,28
6,91040e55-6529-4b0b-bbcc-fa7894e9e623,character,Shadowstrike,Shadowstrike is a highly skilled assassin trai...,28
7,de3dad44-32cc-4938-bcdf-3927e2f33b04,character,Firefly,Firefly is a pyromaniac with a fascination for...,36
8,f644ef0a-b186-4be6-a615-cf3db68b74e6,character,Silverfang,Silverfang is a master martial artist with enh...,32
9,bf2a5b82-7431-4d5b-b656-fd077767227a,character,Ravenstrike,Ravenstrike is a mysterious and enigmatic figu...,28


Now that the characters are embedded, we can query the collection.

In [8]:
villains = query('they are a villain')
villains

,id,type,name,description,age
0,bf2a5b82-7431-4d5b-b656-fd077767227a,character,Ravenstrike,Ravenstrike is a mysterious and enigmatic figu...,28
1,1e779e73-7625-4385-8530-16fe796b6b1a,character,Blazing Fury,Blazing Fury is a fiery and volatile character...,33
2,91040e55-6529-4b0b-bbcc-fa7894e9e623,character,Shadowstrike,Shadowstrike is a highly skilled assassin trai...,28
3,f644ef0a-b186-4be6-a615-cf3db68b74e6,character,Silverfang,Silverfang is a master martial artist with enh...,32
4,400358e0-7249-4b69-932f-1b494b6aca13,character,Nightshade,Nightshade is a mysterious vigilante with the ...,30
5,571338d9-1634-4532-be6c-000142ca4f8f,character,Joker,The Joker is a supervillain and the archenemy ...,42


This compares the query text with the stored objects, returning results that are closest in vector space.

*Note: the effectiveness of embedding queries will depend on what data has been embedded. In this case, ChatGPT will know some details about the generated characters and so does a decent job on this data. For other data, you may find generating synthetic intermediary data to be helpful. E.g. generating `thoughts` and/or `quotes` about a set of documents.*

Because `Collection` extends `pd.DataFrame`, we can use all the usual Pandas methods to filter and sort the results.

In [9]:
villains[villains.age < 30]

,id,type,name,description,age
0,bf2a5b82-7431-4d5b-b656-fd077767227a,character,Ravenstrike,Ravenstrike is a mysterious and enigmatic figu...,28
2,91040e55-6529-4b0b-bbcc-fa7894e9e623,character,Shadowstrike,Shadowstrike is a highly skilled assassin trai...,28


Relationships can be defined by setting the field to a type which subclasses `Entity` (or a list of that type). Internally, this is stored as a query and then loaded when the field is accessed from the database.

In [10]:
class StoryIdea(Entity):
    title: str
    description: str = None
    characters: list[Character] = None

characters = query('they are a villain').sample(3).objects

ideas = prompt(
    'Generate some story ideas',
    input={
        'characters': characters,
    },
    output=[StoryIdea],
).objects

for idea in ideas:
    idea.characters = characters

store(*ideas, collection='story-ideas')
query(collection='story-ideas')

/home/rjl/promptx/venv/lib/python3.10/site-packages/pydantic/json_schema.py:2075: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, json_schema_extra={'embed': False}),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)


,id,type,title,description,characters
0,f141ac9e-52e0-4705-8eb1-114bbcbbfe2f,storyidea,Shadows of Justice,"Nightshade, Ravenstrike, and Silverfang team u...",[{'id': '400358e0-7249-4b69-932f-1b494b6aca13'...
1,dae81a68-df1f-4484-a0bf-bd7827e09b1a,storyidea,City of Shadows,Gotham City is plagued by a string of unexplai...,[{'id': '400358e0-7249-4b69-932f-1b494b6aca13'...
2,80d4020b-b61f-44f7-81bf-fcd3c7fdf18a,storyidea,Shadows of Betrayal,"Nightshade, Ravenstrike, and Silverfang find t...",[{'id': '400358e0-7249-4b69-932f-1b494b6aca13'...


Note that the output is being stored in a collection called `story-ideas`, which is created if it doesn't exist. Previously, all the data we've stored has been in the 'default' collection.

*Collections are widely used internally to represent stored models, templates, prompt history, etc. This provides a consistent interface for accessing and manipulating data.*

So far we've used the default model (GPT-3.5) when generating data, but you can specify a custom model using the `llm=` parameter.

In [11]:
from promptx.models.openai import ChatGPT

gpt4 = ChatGPT(id='gpt4', model='gpt4')

characters = prompt(
    'Generate some characters from the Batman universe',
    output=[Character],
    llm=gpt4,
)

/home/rjl/promptx/venv/lib/python3.10/site-packages/pydantic/json_schema.py:2075: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, json_schema_extra={'embed': False}),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)


You can define any commonly used models, templates, etc, along with defining other settings, by creating a `config.py` file in the root of the project (i.e. adjacent to the `.px/` directory). This file is loaded when the project is initialized and a `setup` function is expected. Here's a simple example that defines a few custom models and a template.

```
# ./config.py

from promptx.models.openai import ChatGPT

gpt4 = ChatGPT(id='gpt4', model='gpt4')

def setup(session):
    session.store(gpt4, collection='models')
```
